In [334]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from sklearn import svm
from sklearn import neighbors
from sklearn import cluster

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [335]:
df = pd.read_csv("/Users/nisarg/Downloads/EV_Market_segmentation/prepared_dataset/102_103_105_107_112.csv",thousands=',')
df2 = pd.read_csv("/Users/nisarg/Downloads/EV_Market_segmentation/prepared_dataset/104_modified.csv",thousands=',')
# thousands to convert "100,000,33" comma seperated number to 10000033 a single int
df = df.drop('Sl. No',axis = 1)
df

,State,Two Wheelers,Three Wheelers,Passenger Cars,Buses,Total in state,No. of EV Chargers Sanctioned,No of RO's where EV Charging Facility available,Total Electric Vehicle,Total Non-Electric Vehicle,Total EV supported by state for adoption and manufacturing
0,Meghalaya,0,0,6,0,6,40.0,3.0,49.0,459001.0,10.0
1,Nagaland,23,0,1,0,24,NaN,2.0,58.0,339129.0,NaN
2,Manipur,35,5,12,0,52,NaN,1.0,586.0,499324.0,140.0
3,Tripura,73,0,8,0,81,NaN,3.0,9262.0,650026.0,750.0
4,Himachal Pradesh,0,0,98,0,98,10.0,13.0,1175.0,1964754.0,446.0
5,Goa,0,0,513,1,514,NaN,17.0,3870.0,1071570.0,614.0
6,Jharkhand,1039,16,655,0,1710,NaN,22.0,16811.0,6486937.0,1874.0
7,Assam,1607,117,151,0,1875,20.0,19.0,64766.0,4677053.0,827.0
8,Chandigarh,1526,0,974,0,2500,70.0,4.0,2812.0,746881.0,154.0
9,Bihar,2830,70,271,0,3171,37.0,26.0,83335.0,10407078.0,5255.0


In [336]:
df.columns

Index(['State', 'Two Wheelers', 'Three Wheelers', 'Passenger Cars', 'Buses',
       'Total in state', 'No. of EV Chargers Sanctioned',
       'No of RO's where EV Charging Facility available',
       'Total Electric Vehicle', 'Total Non-Electric Vehicle',
       'Total EV supported by state for adoption and manufacturing'],
      dtype='object')

### Splitting the dataset

- 1st - Type of vehicles (two wheelers , three wheelers , passenger cars , buses and total in state )
- 2nd - Rest of them ( more broader categories )

In [337]:
df_subset = df[['State', 'No. of EV Chargers Sanctioned',
       "No of RO's where EV Charging Facility available",
       'Total Electric Vehicle', 'Total Non-Electric Vehicle',
       'Total EV supported by state for adoption and manufacturing']]
df_subset_type = df[['State', 'Two Wheelers', 'Three Wheelers', 'Passenger Cars','Total in state']]
df_subset.head()

,State,No. of EV Chargers Sanctioned,No of RO's where EV Charging Facility available,Total Electric Vehicle,Total Non-Electric Vehicle,Total EV supported by state for adoption and manufacturing
0,Meghalaya,40.0,3.0,49.0,459001.0,10.0
1,Nagaland,NaN,2.0,58.0,339129.0,NaN
2,Manipur,NaN,1.0,586.0,499324.0,140.0
3,Tripura,NaN,3.0,9262.0,650026.0,750.0
4,Himachal Pradesh,10.0,13.0,1175.0,1964754.0,446.0


In [338]:
df_subset_type.head()

,State,Two Wheelers,Three Wheelers,Passenger Cars,Total in state
0,Meghalaya,0,0,6,6
1,Nagaland,23,0,1,24
2,Manipur,35,5,12,52
3,Tripura,73,0,8,81
4,Himachal Pradesh,0,0,98,98


In [339]:
df2

,Category,Segment,2013-14,2014-15
0,Passenger Vehicles (PVs),Passenger Cars,2519281.0,2590917.0
1,Passenger Vehicles (PVs),Multi-Utility Vehicles,568692.0,629255.0
2,Passenger Vehicles (PVs),Total Passenger Vehicles (PVs),3087973.0,3220172.0
3,Commercial Vehicles - M & HCVs,Passenger Carriers,41175.0,49360.0
4,Commercial Vehicles - M & HCVs,Goods Carriers,180381.0,219193.0
5,Commercial Vehicles - M & HCVs,Total M & HCVs,221556.0,268553.0
6,Commercial Vehicles - LCVs,Passenger Carriers,45136.0,46569.0
7,Commercial Vehicles - LCVs,Goods Carriers,432343.0,381961.0
8,Commercial Vehicles - LCVs,Total LCVs,477479.0,428530.0
9,Commercial Vehicles (CVs),Total Commercial Vehicles (CVs),699035.0,697083.0


In [340]:
df_subset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 6 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   State                                                       26 non-null     object 
 1   No. of EV Chargers Sanctioned                               20 non-null     float64
 2   No of RO's where EV Charging Facility available             25 non-null     float64
 3   Total Electric Vehicle                                      23 non-null     float64
 4   Total Non-Electric Vehicle                                  23 non-null     float64
 5   Total EV supported by state for adoption and manufacturing  25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.3+ KB


### Missing values

In [341]:
df_subset.isna().sum()

State                                                         0
No. of EV Chargers Sanctioned                                 6
No of RO's where EV Charging Facility available               1
Total Electric Vehicle                                        3
Total Non-Electric Vehicle                                    3
Total EV supported by state for adoption and manufacturing    1
dtype: int64

### Lets examine the dataset in Two phases

1. Govt support 
    - No. of EV chargers Sanctioned
    - Total EV supported by state for adoption and manufacturing
2. Public data
    - Total Electric Vehicle
    - Total Non-Electric Vehicle
    - No of RO's where EV Charging Facility available

In [342]:
df_subset[df['No. of EV Chargers Sanctioned'].isna()]

,State,No. of EV Chargers Sanctioned,No of RO's where EV Charging Facility available,Total Electric Vehicle,Total Non-Electric Vehicle,Total EV supported by state for adoption and manufacturing
1,Nagaland,NaN,2.0,58.0,339129.0,NaN
2,Manipur,NaN,1.0,586.0,499324.0,140.0
3,Tripura,NaN,3.0,9262.0,650026.0,750.0
5,Goa,NaN,17.0,3870.0,1071570.0,614.0
6,Jharkhand,NaN,22.0,16811.0,6486937.0,1874.0
13,Punjab,NaN,41.0,14804.0,12463019.0,2977.0


In [343]:
df_subset[df['Total EV supported by state for adoption and manufacturing'].isna()]

,State,No. of EV Chargers Sanctioned,No of RO's where EV Charging Facility available,Total Electric Vehicle,Total Non-Electric Vehicle,Total EV supported by state for adoption and manufacturing
1,Nagaland,NaN,2.0,58.0,339129.0,NaN


<b> The reason of NaN values in above two fields might be that theres no policies regarding the EV chargers or EV support , so it will be appropriate if we fill all NaNs here with 0.

In [344]:
df_subset['No. of EV Chargers Sanctioned'] = df_subset['No. of EV Chargers Sanctioned'].fillna(0)
df_subset['Total EV supported by state for adoption and manufacturing'] = df_subset['Total EV supported by state for adoption and manufacturing'].fillna(0)

/var/folders/4j/_ythdyc10c14y246dzp2mj4m0000gn/T/ipykernel_60243/3463724593.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/4j/_ythdyc10c14y246dzp2mj4m0000gn/T/ipykernel_60243/3463724593.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [345]:
df_subset.isna().sum()

State                                                         0
No. of EV Chargers Sanctioned                                 0
No of RO's where EV Charging Facility available               1
Total Electric Vehicle                                        3
Total Non-Electric Vehicle                                    3
Total EV supported by state for adoption and manufacturing    0
dtype: int64

In [346]:
df_subset[df["No of RO's where EV Charging Facility available"].isna()]

,State,No. of EV Chargers Sanctioned,No of RO's where EV Charging Facility available,Total Electric Vehicle,Total Non-Electric Vehicle,Total EV supported by state for adoption and manufacturing
25,Maharashtra,317.0,NaN,116646.0,31058990.0,20955.0


In [347]:
df_subset["No of RO's where EV Charging Facility available"] = df_subset["No of RO's where EV Charging Facility available"].fillna(90)

/var/folders/4j/_ythdyc10c14y246dzp2mj4m0000gn/T/ipykernel_60243/1242122970.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [348]:
df_subset[["No of RO's where EV Charging Facility available","State"]]

,No of RO's where EV Charging Facility available,State
0,3.0,Meghalaya
1,2.0,Nagaland
2,1.0,Manipur
3,3.0,Tripura
4,13.0,Himachal Pradesh
5,17.0,Goa
6,22.0,Jharkhand
7,19.0,Assam
8,4.0,Chandigarh
9,26.0,Bihar


In [349]:
df_subset[df['Total Non-Electric Vehicle'].isna()]

,State,No. of EV Chargers Sanctioned,No of RO's where EV Charging Facility available,Total Electric Vehicle,Total Non-Electric Vehicle,Total EV supported by state for adoption and manufacturing
12,Madhya Pradesh,235.0,167.0,NaN,NaN,6306.0
14,Telangana,138.0,112.0,NaN,NaN,6316.0
15,Andhra Pradesh,266.0,65.0,NaN,NaN,6393.0


In [350]:
df_subset[df["Total Electric Vehicle"].isna()]

,State,No. of EV Chargers Sanctioned,No of RO's where EV Charging Facility available,Total Electric Vehicle,Total Non-Electric Vehicle,Total EV supported by state for adoption and manufacturing
12,Madhya Pradesh,235.0,167.0,NaN,NaN,6306.0
14,Telangana,138.0,112.0,NaN,NaN,6316.0
15,Andhra Pradesh,266.0,65.0,NaN,NaN,6393.0


There big states are missing values , this has to be a error or no data available for the same.

We will try to find the data from the net
                       Electric      Non-Electric
    Telangana       -   46937          15066192
    Madhya Pradesh  -   NA              NA
    Andhra pradesh  -   NA              NA
for the rest we won't consider MP and AP for further analyses by creating a subset and removing them.

In [351]:
df_state_wise_subset = df_subset.set_index("State")

In [352]:
df_state_wise_subset

,No. of EV Chargers Sanctioned,No of RO's where EV Charging Facility available,Total Electric Vehicle,Total Non-Electric Vehicle,Total EV supported by state for adoption and manufacturing
State,,,,,
Meghalaya,40.0,3.0,49.0,459001.0,10.0
Nagaland,0.0,2.0,58.0,339129.0,0.0
Manipur,0.0,1.0,586.0,499324.0,140.0
Tripura,0.0,3.0,9262.0,650026.0,750.0
Himachal Pradesh,10.0,13.0,1175.0,1964754.0,446.0
Goa,0.0,17.0,3870.0,1071570.0,614.0
Jharkhand,0.0,22.0,16811.0,6486937.0,1874.0
Assam,20.0,19.0,64766.0,4677053.0,827.0
Chandigarh,70.0,4.0,2812.0,746881.0,154.0


In [353]:
df_state_wise_subset.loc["Telangana",["Total Electric Vehicle","Total Non-Electric Vehicle"]] = [46937,15066192]

In [354]:
df_state_wise_subset.loc['Telangana']

No. of EV Chargers Sanctioned                                      138.0
No of RO's where EV Charging Facility available                    112.0
Total Electric Vehicle                                           46937.0
Total Non-Electric Vehicle                                    15066192.0
Total EV supported by state for adoption and manufacturing        6316.0
Name: Telangana, dtype: float64

In [355]:
df_state_wise_subset.columns

Index(['No. of EV Chargers Sanctioned',
       'No of RO's where EV Charging Facility available',
       'Total Electric Vehicle', 'Total Non-Electric Vehicle',
       'Total EV supported by state for adoption and manufacturing'],
      dtype='object')

In [356]:
df_charging_subset = df_state_wise_subset[["No. of EV Chargers Sanctioned","No of RO's where EV Charging Facility available"]]
df_charging_subset

,No. of EV Chargers Sanctioned,No of RO's where EV Charging Facility available
State,,
Meghalaya,40.0,3.0
Nagaland,0.0,2.0
Manipur,0.0,1.0
Tripura,0.0,3.0
Himachal Pradesh,10.0,13.0
Goa,0.0,17.0
Jharkhand,0.0,22.0
Assam,20.0,19.0
Chandigarh,70.0,4.0


In [357]:
df_ev_sale_subset = df_state_wise_subset[['Total Electric Vehicle', 'Total Non-Electric Vehicle',
       'Total EV supported by state for adoption and manufacturing']]
df_ev_sale_subset.drop(['Andhra Pradesh','Madhya Pradesh'],axis = 0,inplace = True)
df_ev_sale_subset

/var/folders/4j/_ythdyc10c14y246dzp2mj4m0000gn/T/ipykernel_60243/2366790060.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Total Electric Vehicle,Total Non-Electric Vehicle,Total EV supported by state for adoption and manufacturing
State,,,
Meghalaya,49.0,459001.0,10.0
Nagaland,58.0,339129.0,0.0
Manipur,586.0,499324.0,140.0
Tripura,9262.0,650026.0,750.0
Himachal Pradesh,1175.0,1964754.0,446.0
Goa,3870.0,1071570.0,614.0
Jharkhand,16811.0,6486937.0,1874.0
Assam,64766.0,4677053.0,827.0
Chandigarh,2812.0,746881.0,154.0


In [358]:
df_state_wise_subset['Total Electric Vehicle'] = df_state_wise_subset['Total Electric Vehicle'].fillna(83335)

df_state_wise_subset['Total Non-Electric Vehicle'] = df_state_wise_subset['Total Non-Electric Vehicle'].fillna(10407078)

## Performing EDA
final dfs :

 - df_ev_sale_subset
 - df_charging_subset
 - df_state_wise_subset    { contains null value filled as 83335 based on bihars value }

In [359]:
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [360]:
def plot_pie(dataset:pd.DataFrame,column:str):
    fig = px.pie(df_ev_sale_subset, values=df_ev_sale_subset[column], names=df_ev_sale_subset.index,
                 color_discrete_sequence=pal_)
    fig.update_layout(width = 700, height = 700,
                      margin = dict(t=0, l=0, r=0, b=0))
    fig.update_traces(textfont_size=16)
    fig.show()

In [361]:
plot_pie(df_ev_sale_subset,"Total Electric Vehicle")

In [362]:
plot_pie(df_ev_sale_subset,"Total Non-Electric Vehicle")

In [363]:
plot_pie(df_ev_sale_subset,"Total EV supported by state for adoption and manufacturing")

In [364]:
standard_ev_sale_subset = StandardScaler().fit_transform(df_ev_sale_subset)

In [365]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(standard_ev_sale_subset)
principal_df_ev_sale_subset = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

In [368]:
principal_df_ev_sale_subset.set_index(df_ev_sale_subset.index)

,principal component 1,principal component 2
State,,
Meghalaya,-1.515046,0.008143
Nagaland,-1.522385,0.009018
Manipur,-1.500197,0.002663
Tripura,-1.385199,0.036827
Himachal Pradesh,-1.393155,-0.016384
Goa,-1.411843,-0.004289
Jharkhand,-0.924857,0.019395
Assam,-0.724886,0.571680
Chandigarh,-1.468153,0.023240


AttributeError: 'DataFrame' object has no attribute 'explained_variance_ratio_'